In [11]:
# https://wikidocs.net/60691

# 02. IMDB 리뷰 감성 분류하기(IMDB Movie Review Sentiment Analysis)

참고 논문: http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf

In [12]:
# 영화 사이트 IMDB의 리뷰 데이터를 이용해 1 긍정 , 0 부정을 분류해보자

In [35]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext.legacy import data, datasets
import random

In [36]:
# 랜덤시드 고정
SEED = 5
random.seed(SEED)
torch.manual_seed(SEED)

In [37]:
# 하이퍼파라미터 변수
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 10

In [89]:
# 디바이스 설정
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("cpu와 cuda 중 다음 기기로 학습함:", DEVICE)

cpu와 cuda 중 다음 기기로 학습함: cuda


### 2. 토치텍스트를 이용한 전처리
---

In [90]:
# 1. 데이터 로드하기 : torchtext.data
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
# sequential: 순차적 데이터
# batch_first: 신경망에 입력되는 텐서의 첫번째 차원값이 batch_size가 되도록
# lower: 입력되는 모든 알파벳이 소문자가 되도록
LABEL = data.Field(sequential=False, batch_first=True)

In [91]:
# 2. 데이터 로드 및 분할하기 : torchtext.datasets

# 전체 데이터를 훈련, 테스트로 나누기(8:2)
path = 'D:/chchdata/dataset/imdb_data'
trainset, testset = datasets.IMDB.splits(root=path, text_field = TEXT, label_field= LABEL)

In [92]:
print('trainset의 구성 요소 출력: ', trainset.fields)
print('testset의 구성 요소 출력: ', testset.fields)

trainset의 구성 요소 출력:  {'text': <torchtext.legacy.data.field.Field object at 0x0000024E5D273160>, 'label': <torchtext.legacy.data.field.Field object at 0x0000024E5D273100>}
testset의 구성 요소 출력:  {'text': <torchtext.legacy.data.field.Field object at 0x0000024E5D273160>, 'label': <torchtext.legacy.data.field.Field object at 0x0000024E5D273100>}


In [93]:
# 첫번째 훈련 샘플과 해당 샘플에 대한 레이블을 함께 출력
print(vars(trainset[0]))

{'text': ['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy.', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life,', 'such', 'as', '"teachers".', 'my', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'bromwell', "high's", 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', '"teachers".', 'the', 'scramble', 'to', 'survive', 'financially,', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', "teachers'", 'pomp,', 'the', 'pettiness', 'of', 'the', 'whole', 'situation,', 'all', 'remind', 'me', 'of', 'the', 'schools', 'i', 'knew', 'and', 'their', 'students.', 'when', 'i', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school,', 'i', 'immediately', 'recalled', '.........', 'at', '..........', 'high.', 'a', 'classic', 'line:', 'inspector:', "i'm", 'here', 'to', 'sack', 'one', 'of', '

In [94]:
# 3. 단어 집합 만들기
# 중복을 제거한 총 단어 집합 만들기

TEXT.build_vocab(trainset, min_freq = 5) # 최소 5번 이상 등장한 것만 추가
LABEL.build_vocab(trainset)

In [95]:
vocab_size = len(TEXT.vocab)
n_classes = 2
print('단어 집합의 크기 : {}'.format(vocab_size))
print('클래스의 개수 : {}'.format(n_classes))

단어 집합의 크기 : 46159
클래스의 개수 : 2


In [47]:
# 각 단어와 정수가 저장되어 있는 dict 출력해보자
print(TEXT.vocab.stoi)

In [96]:
# 4. 데이터 로더 만들기

# 훈련, 테스트를 나누었으니 이번에는 검증까지 분리하자
trainset, valset = trainset.split(split_ratio=0.8)

In [97]:
# 텍스트 배치 처리 + 단어 인덱스 번호 대체 를 제공하는 BucketIterator를 사용하자
train_iter, val_iter, test_iter = data.BucketIterator.splits((trainset, valset, testset)
                                                            , batch_size = BATCH_SIZE   # 64
                                                            , shuffle=True
                                                            , repeat=False) # 모든 에폭에 반복할지 여부

In [98]:
# 64 batch size로 묶인 데이터의 형태 확인
print('훈련 데이터의 미니 배치의 개수 : {}'.format(len(train_iter)))
print('테스트 데이터의 미니 배치의 개수 : {}'.format(len(test_iter)))
print('검증 데이터의 미니 배치의 개수 : {}'.format(len(val_iter)))

훈련 데이터의 미니 배치의 개수 : 313
테스트 데이터의 미니 배치의 개수 : 391
검증 데이터의 미니 배치의 개수 : 79


In [86]:
# 첫번째 미니 배치 크기 확인
batch = next(iter(train_iter))
print(batch.text.shape)

# 프린트 할 때 마다 train_iter에서 하나씩 빠지므로 주의

torch.Size([64, 807])


In [145]:
# 3. RNN 모델 구현하기

class GRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(GRU, self).__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.dropout = nn.Dropout(dropout_p)
        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        x = self.embed(x)
        # 첫번째 hidden state를 0벡터로 초기화
        h_0 = self._init_state(batch_size=x.size(0))
        # GRU의 리턴값은 (배치 크기, 시퀀스 길이, 은닉 상태의 크기)
        x, _ = self.gru(x, h_0)
        # (배치큭, 은닉 상태의 크기)의 텐서로 크기가 변경됨. 즉, 마지막 time_step의 은닉 상태만 가져옴
        h_t = x[:,-1,:]
        self.dropout(h_t)
        # (배치크기, 은닉 상태의 크기) -> 배치 크기, 출력층의 크기) 로 변환
        logit = self.out(h_t)
        return logit

    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [146]:
model = GRU(n_layers = 1
            , hidden_dim = 256
            , n_vocab = vocab_size
            , embed_dim = 28
            , n_classes = n_classes
            , dropout_p = 0.5).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [147]:
# 모델을 이용한 훈련 함수

def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)  # 1과 2였던 label 값에서 모두 1씩 빼고 대체하여 레이블 값을 0과 1로 변환
        optimizer.zero_grad()

        logit = model(x)
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()

In [170]:
# 모델 평가 함수
def evaluate(model, val_iter):
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1) # 레이블 값을 0과 1로 변환
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction='sum')
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy

In [171]:
best_val_loss = None
for e in range(1, EPOCHS+1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)

    print("[Epoch: %d] val loss : %5.2f | val accuracy : %5.2f" % (e, val_loss, val_accuracy))

    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("D:/chchdata/h5/snapshot"):
            os.makedirs("D:/chchdata/h5/snapshot")
        torch.save(model.state_dict(), 'D:/chchdata/h5/snapshot/txtclassification.pt')
        best_val_loss = val_loss

[Epoch: 1] val loss :  0.69 | val accuracy : 49.96
[Epoch: 2] val loss :  0.69 | val accuracy : 50.22
[Epoch: 3] val loss :  0.69 | val accuracy : 49.18
[Epoch: 4] val loss :  0.69 | val accuracy : 49.32
[Epoch: 5] val loss :  0.69 | val accuracy : 51.16
[Epoch: 6] val loss :  0.69 | val accuracy : 50.36
[Epoch: 7] val loss :  0.70 | val accuracy : 50.28
[Epoch: 8] val loss :  0.72 | val accuracy : 50.02
[Epoch: 9] val loss :  0.73 | val accuracy : 51.32
[Epoch: 10] val loss :  0.74 | val accuracy : 49.74


In [172]:
model.load_state_dict(torch.load('D:/chchdata/h5/snapshot/txtclassification.pt'))
test_loss, test_acc = evaluate(model, test_iter)
print('테스트 오차: %5.2f | 테스트 정확도: %5.2f' % (test_loss, test_acc))

테스트 오차:  0.69 | 테스트 정확도: 47.78


In [173]:
# 4. 마지막 time step의 hidden state 가져오는 것 이해하기

# GRU의 마지막 time_step의 hidden_state를 가져오는 코드를 임의의 텐서를 사용해 실습해보자
import torch
inputs = torch.rand(3, 4, 5) # 임의의 3차원 텐서 생성

print('텐서의 크기 :',inputs.shape)

텐서의 크기 : torch.Size([3, 4, 5])


In [174]:
# 텐서값을 출력해보자
print(inputs)

tensor([[[0.0235, 0.4590, 0.9496, 0.8781, 0.8022],
         [0.4896, 0.3085, 0.3007, 0.0315, 0.9417],
         [0.3339, 0.0917, 0.2220, 0.6460, 0.9839],
         [0.3196, 0.7640, 0.0758, 0.0167, 0.1228]],

        [[0.6639, 0.6345, 0.9903, 0.1532, 0.9122],
         [0.9826, 0.3369, 0.0994, 0.0434, 0.2157],
         [0.8979, 0.3712, 0.8974, 0.6721, 0.4596],
         [0.1289, 0.7171, 0.7642, 0.5292, 0.2155]],

        [[0.3976, 0.9681, 0.6180, 0.6289, 0.9304],
         [0.6978, 0.1229, 0.1887, 0.3612, 0.2288],
         [0.2392, 0.1942, 0.1158, 0.8707, 0.6355],
         [0.9193, 0.0402, 0.2062, 0.8157, 0.7216]]])


In [175]:
# 이제 [:, -1, :] 연산을 해보자
print(inputs[:, -1, :])

tensor([[0.3196, 0.7640, 0.0758, 0.0167, 0.1228],
        [0.1289, 0.7171, 0.7642, 0.5292, 0.2155],
        [0.9193, 0.0402, 0.2062, 0.8157, 0.7216]])


In [2]:
# 0차원의 각 마지막 줄만 가져왔다

# 바뀐 형태도 보자
print('텐서의 크기 :',inputs[:, -1, :].shape) 